### Q1. Running Elastic

In [ ]:
# # Run Elastic Search 8.17.6

# docker run -it \
#     --rm \
#     --name elasticsearch \
#     -m 4GB \
#     -p 9200:9200 \
#     -p 9300:9300 \
#     -e "discovery.type=single-node" \
#     -e "xpack.security.enabled=false" \
#     docker.elastic.co/elasticsearch/elasticsearch:8.17.6

In [1]:
!curl localhost:9200

{
  "name" : "e07a906cb8fd",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "91ltceNkQYuI8trMz8luzw",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
# Q: What's the version.build_hash value?

# A:  dbcbbbd0bc4924cfeb28929dc05d82d662c527b7

In [2]:
# Getting the data
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing the data

In [ ]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

es_client = Elasticsearch('http://127.0.0.1:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

# add data to elastic
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:05<00:00, 174.85it/s]


In [ ]:
# Q: Which function do you use for adding your data to elastic?

# A: index

### Q3. Searching

In [10]:
# Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        # result_docs.append(hit['_source'])
        result_docs.append(hit)
    
    return result_docs

In [11]:
query = "How do execute a command on a Kubernetes pod?"

response = elastic_search(query)


In [12]:
response

[{'_index': 'course-questions',
  '_id': 'rF7ecZcBNsD8-cVEODuS',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'O17ecZcBNsD8-cVEOzwu',
  '_score': 35.433445,
  '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'zF7ecZcBNsD8-cVEOTsh',
  '_score': 33.70974,
  '_source': {'text': 'You 

In [13]:
# Q: What's the score for the top ranking result?

print("The score for the top ranking result is", response[0]['_score'])

The score for the top ranking result is 44.50556


### Q4. Filtering

In [14]:
# This time we are only interested in questions from machine-learning-zoomcamp.

def elastic_search2(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        # result_docs.append(hit['_source'])
        result_docs.append(hit)
    
    return result_docs

In [15]:
query2 = "How do copy a file to a Docker container?"

response2 = elastic_search2(query2)

response2

[{'_index': 'course-questions',
  '_id': 'rF7ecZcBNsD8-cVEODuS',
  '_score': 73.38676,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'y17ecZcBNsD8-cVEOTsd',
  '_score': 66.688705,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

In [18]:
# Q: Return 3 results. What's the 3rd question returned by the search engine?

print("The 3rd question returned by the search engine:", response2[2]['_source']['question'])

The 3rd question returned by the search engine: How do I copy files from a different folder into docker container’s working directory?


### Q5. Building a prompt

In [19]:
# Take the records returned from Elasticsearch in Q4 and use this template to build the context.

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [20]:
search_results = [hit['_source'] for hit in response2]
prompt = build_prompt(query2, search_results)

In [21]:
# What's the length of the resulting prompt? (use the len function)

print(len(prompt))

1446


### Q6. Tokens

In [24]:
import tiktoken

# Load encoding for gpt-4o
encoding = tiktoken.encoding_for_model("gpt-4o")

In [25]:
# Encode the prompt into token IDs
token_ids = encoding.encode(prompt)

# Number of tokens
num_tokens = len(token_ids)

In [26]:
# Q: Use the encode function. How many tokens does our prompt have?

print(f"Number of tokens: {num_tokens}")

Number of tokens: 320
